<a href="https://colab.research.google.com/github/leotuni/Nicolas-stuff/blob/master/Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Corona data cause I'm bored 
I am importing all the data from the GITHUB REPOSITORY CSSEGISandData, which is run by John Hopkins University. 

In [0]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

In [0]:
#@title Import and Option to show print more data 
import pandas as pd 
import glob
#plotting 
import plotly.express as px


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', frame.shape[0]+1)

I import all the files and concatenate them into one dataframe (**frame**) 

I then: 

0.   they used Mainlad china and China, so replaced mainland china to china 
1.   change **Last Update** to Date format, and sort by Date
2.   Fill the NaN of **State/Province** with the **Country/Region** values to have a full collumn
3.   Subsett by **Last Update** and **State/Province** and drop all Duplicates

The reason there is Duplicates is that many coutries didn't report new data every day, so a file from date: 1/2/2020 would have the data from Sweden of the 31/1/2020 if There is no new **Last Update**



In [39]:
#@title Import files and Prepare data 

path = r'/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
all_files = glob.glob(path + "/*.csv") #collect all files in one
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['file']=filename[-14:]
    li.append(df)
#Make one file and sort
frame = pd.concat(li, axis=0, ignore_index=True, sort=False) #one dataframe 
frame['Country/Region'] = frame['Country/Region'].str.replace('Mainland China','China')
frame['Last Update'] = pd.to_datetime(frame['Last Update'])
frame = frame.sort_values(by="Last Update")
frame['Province/State'].fillna(frame['Country/Region'], inplace=True) #putting the Coutry in teh NaN of Province for data cleanup 

frame = frame.assign(day=frame['Last Update'].dt.date).\
          drop_duplicates(subset=(['day', 'Province/State'])).drop('day',axis=1)
frame['Deaths'].fillna(0, inplace=True)
frame['Recovered'].fillna(0, inplace=True)

frame.head()


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,file
1316,Sichuan,China,2020-01-22 17:00:00,5.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1318,Tianjin,China,2020-01-22 17:00:00,4.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1317,Taiwan,Taiwan,2020-01-22 17:00:00,1.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1315,Shanxi,China,2020-01-22 17:00:00,1.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1314,Shanghai,China,2020-01-22 17:00:00,9.0,0.0,0.0,NaN,NaN,01-22-2020.csv


In [54]:
frame['Total_deaths_by_country'] = frame.groupby([frame['Last Update'].dt.date ,frame['Country/Region']])['Deaths'].transform('sum')
frame['Total_Cases_by_country'] = frame.groupby([frame['Last Update'].dt.date ,frame['Country/Region']])['Confirmed'].transform('sum')
frame['Death Percentage'] = frame['Total_deaths_by_country']/ frame['Total_Cases_by_country'] *100
frame['Death Percentage']=frame['Death Percentage'].round(2)
filterEurope= frame['Country/Region'].isin(['Italy', 'Germany', 'France', 'Spain', 'Denmark', 'Holland','Switzerland','Norway','Czech Republic','Austria','UK','Sweden','Belgium','Portugal','Poland'])

# Hubei=frame[frame['Province/State']=='Hubei']
frame['Country/Region'].unique()

array(['China', 'Taiwan', 'Macau', 'Hong Kong', 'US', 'South Korea',
       'Japan', 'Thailand', 'Singapore', 'Malaysia', 'Vietnam', 'Mexico',
       'Brazil', 'Colombia', 'Philippines', 'Australia', 'France',
       'Nepal', 'Canada', 'Sri Lanka', 'Cambodia', 'Ivory Coast',
       'Germany', 'United Arab Emirates', 'Finland', 'India', 'Italy',
       'Russia', 'UK', 'Sweden', 'Spain', 'Belgium', 'Others', 'Egypt',
       'Iran', 'Israel', 'Lebanon', 'Iraq', 'Afghanistan', 'Bahrain',
       'Kuwait', 'Oman', 'Austria', 'Croatia', 'Switzerland', 'Algeria',
       'Greece', 'North Macedonia', 'Georgia', 'Norway', 'Romania',
       'Pakistan', 'Denmark', 'Estonia', 'San Marino', 'Netherlands',
       'North Ireland', ' Azerbaijan', 'Iceland', 'Lithuania',
       'New Zealand', 'Nigeria', 'Belarus', 'Monaco', 'Qatar',
       'Luxembourg', 'Ireland', 'Azerbaijan', 'Czech Republic',
       'Dominican Republic', 'Armenia', 'Ecuador', 'Indonesia', 'Latvia',
       'Andorra', 'Senegal', 'Morocc

In [41]:
#@title Total deaths by country 
fig = px.line(frame, x='Last Update', y='Total_deaths_by_country', color='Country/Region', title=' Total deaths by country')
fig.show()

In [42]:
#@title Total cases by country 
fig = px.line(frame, x='Last Update', y='Total_Cases_by_country', color='Country/Region', title=' Total cases by country')
fig.show()

# Death Percentage vs Number of Confirmed cases 

*   Italy has a very high death rate compared with the otehr countries, perhaps beccuase it is further
*   Germany stands out as a particularly low death rate


In [57]:
#@title Death Percentage by filtercoutry
fig = px.line(frame[filterEurope], x='Last Update', y='Death Percentage', color='Country/Region', title='Death Percentage vs Total cases ')
fig.show()